In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
import os
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers

from bert.dataset import create_masked_input_dataset
from bert.layers import (PositionEmbedding, Attention, Transformer, TokenEmbedding, Bias,
                         gelu, masked_sparse_cross_entropy_loss, BERTLearningRateScheduler,
                         initializer, InverseSquareRootSchedule)

training_data = create_masked_input_dataset(
    language_model_path='sentencepiece_models/uniparc_10M_8000.model',
    sequence_path='/projects/bpms/pstjohn/uniparc/sequences_train.txt',
    max_sequence_length=512,
    batch_size=20,
    buffer_size=1024,
    vocab_size=8000,
    mask_index=4,
    vocab_start=5,
    fix_sequence_length=True,
)

training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)


valid_data = create_masked_input_dataset(
    language_model_path='sentencepiece_models/uniparc_10M_8000.model',
    sequence_path='/projects/bpms/pstjohn/uniparc/sequences_valid.txt',
    max_sequence_length=512,
    batch_size=20,
    buffer_size=1024,
    vocab_size=8000,
    mask_index=4,
    vocab_start=5,
    fix_sequence_length=True)

valid_data.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: (((None, 512), (None, 512)), (None, 512)), types: ((tf.int32, tf.bool), tf.int32)>

In [4]:
vocab_size = 8000
max_seq_len = 512
embedding_dimension = 32
model_dimension = 64
num_attention_heads = model_dimension // 16
num_transformer_layers = 12
dropout_rate = 0.1

# mirrored_strategy = tf.distribute.MirroredStrategy()
# with mirrored_strategy.scope():

inputs = layers.Input(shape=(max_seq_len,), dtype=tf.int32, batch_size=None)
input_mask = layers.Input(shape=(max_seq_len,), dtype=tf.bool, batch_size=None)

token_embedding_layer = TokenEmbedding(
    vocab_size, embedding_dimension, embeddings_initializer=initializer(), mask_zero=True)
token_embeddings = token_embedding_layer(inputs)
position_embeddings = PositionEmbedding(
    max_seq_len + 1, embedding_dimension, embeddings_initializer=initializer(),
    mask_zero=True)(inputs)

embeddings = layers.Add()([token_embeddings, position_embeddings])
embeddings = layers.Dense(model_dimension)(embeddings)

transformer = Transformer(num_attention_heads)
for i in range(num_transformer_layers):
    embeddings = transformer(embeddings)

out = layers.Dense(embedding_dimension, activation=gelu, kernel_initializer=initializer())(embeddings)
out = token_embedding_layer(out, transpose=True)
out = Bias()([out, input_mask])

model = tf.keras.Model([inputs, input_mask], [out], name='model')
model.summary()

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction=tf.keras.losses.Reduction.NONE),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    optimizer=tfa.optimizers.AdamW(weight_decay=0.01, learning_rate=1E-3))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
token_embedding (TokenEmbedding multiple             256000      input_1[0][0]                    
                                                                 dense_1[0][0]                    
__________________________________________________________________________________________________
position_embedding (PositionEmb (None, 512, 32)      16416       input_1[0][0]                    
__________________________________________________________________________________________________
add (Add)                       (None, 512, 32)      0           token_embedding[0][0]        

In [5]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

callbacks = [
#    tf.keras.callbacks.TensorBoard(log_dir='./logs', profile_batch=0),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix),
    tf.keras.callbacks.CSVLogger('log.csv'),    
    InverseSquareRootSchedule(learning_rate=1E-3, warmup_updates=1000)
]

In [6]:
model.fit(training_data, steps_per_epoch=100, epochs=10, verbose=1,
          validation_data=valid_data, validation_steps=10,
          callbacks=callbacks)

Train for 100 steps, validate for 10 steps
Epoch 1/10


/home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


 99/100 [============================>.] - ETA: 0s - loss: 0.4136 - sparse_categorical_accuracy: 0.0081WARNING:tensorflow:From /home/pstjohn/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_1/assets
100/100 [==============================] - 68s 680ms/step - loss: 0.4133 - sparse_categorical_accuracy: 0.0084 - val_loss: 0.4005 - val_sparse_categorical_accuracy: 0.0219
Epoch 2/10
 78/100 [======================>.......] - ETA: 3s - loss: 0.4234 - sparse_categorical_accuracy: 0.0204INFO:tensorflow:Assets written to: ./training_checkpoints/ckpt_2/assets


KeyError: 'val_loss'

In [7]:
tf.keras.models.load_model('training_checkpoints/ckpt_2/')

In [ ]:
import os
import json

In [ ]:
with open('gpuhosts.txt') as file:
    hosts = file.readlines()

In [ ]:
port = 1234

In [ ]:
[f"{worker.strip()}:{port}" for worker in hosts]

In [ ]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["host1:port", "host2:port", "host3:port"],
    },
   "task": {"type": "worker", "index": 1}
})